In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

train_data=datasets.MNIST(root='./data',transform=ToTensor(),train=True,download=True)
test_data=datasets.FashionMNIST(root='./data',transform=ToTensor(),train=False,download=True)

batch_size=4
train_loader=DataLoader(train_data,shuffle=True,batch_size=batch_size)
test_loader=DataLoader(test_data,shuffle=False,batch_size=batch_size)

image,label = next(iter(train_loader))
plt.imshow(image[0].resize(28,28),cmap='gray')

class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel,self).__init__()
    self.net=nn.Sequential(
        nn.Conv2d(1,64,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2),
        nn.Conv2d(64,128,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2),
        nn.Conv2d(128,64,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2)
    )
    self.classification_head=nn.Sequential(
        nn.Linear(64,20,bias=True),
        nn.Sigmoid(),
        nn.Linear(20,10,bias=True)
    )
  def forward(self,x):
    conv_output=self.net(x)
    x=self.classification_head(conv_output.view(batch_size,-1))
    return x

device='cuda'
model=CNNModel().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn=torch.nn.CrossEntropyLoss()

#total number of parameters
total_param=sum([p.numel() for p in model.parameters() if p.requires_grad])
total_param

def train_one_epoch():
  total_loss=0
  for i,(input,label) in enumerate(train_loader):
    input,label=input.to(device),label.to(device)
    optimizer.zero_grad()
    output=model(input)
    loss=loss_fn(output,label)
    loss.backward()
    optimizer.step()
    loss+=loss.item()
  return loss/len(train_loader)

for i in range(10):
  model.train()
  res=train_one_epoch()
  print(res)

print('Models state dict: ')
for keys in model.state_dict().keys():
  print(keys,'/t',model.state_dict()[keys])

print('optimizer state dict')
for keys in optimizer.state_dict().keys():
  print(keys,'\t',optimizer.state_dict()[keys])

torch.save(model,".model.pt")
model=CNNModel()
model=torch.load(".model.pt")
model.to(device)

count=0
correct=0
model.eval()
for i,(input,label) in enumerate(test_loader):
  input=input.to(device)
  label=label.to(device)
  output=model(input)
  _,predicted=torch.max(output,1)
  count+=label.size(0)
  correct+=(predicted==label).sum()
acc=(correct/count)*100
print(acc)

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

batch_size=4

device='cuda'

train_set=datasets.MNIST(root='./data',transform=ToTensor(),train=True,download=True)
test_set=datasets.FashionMNIST(root='./data',transform=ToTensor(),train=False,download=True)

train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_set,batch_size=batch_size,shuffle=False)

class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.net=nn.Sequential(
        nn.Conv2d(1,64,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2),
        nn.Conv2d(64,128,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2),
        nn.Conv2d(128,64,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2)
    )
    self.activation_head=nn.Sequential(
        nn.Linear(64,20,bias=True),
        nn.Sigmoid(),
        nn.Linear(20,10,bias=True)
    )
  def forward(self,x):
    y=self.net(x)
    return self.activation_head(y.view(batch_size,-1))

model=CNN().to(device)
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)
loss_fn=nn.CrossEntropyLoss()


breaker=0
for i in range(10):
  breaker+=1
  loss_list=[]
  for _,(input,label) in enumerate(train_loader):
    input=input.to(device)
    label=label.to(device)
    optimizer.zero_grad()
    output=model(input)
    loss=loss_fn(output,label)
    loss.backward()
    optimizer.step()
    loss_list.append(loss.item())
  if breaker==2:
    break



In [ ]:
checkpoint={'loss':loss_list[-1],'epoch':breaker,'model_state_dict':model.state_dict(),'optimizer_state_dict':optimizer.state_dict()}
torch.save(checkpoint,'./checkpoint.pt')
model=CNN().to(device)
cp=torch.load('./checkpoint.pt')
optimizer=nn.CrossEntropyLoss()
optimizer.load_state_dict(cp['optimizer_state_dict'])
epoch=cp['epoch']
loss=cp['loss']

In [ ]:
loss

2.0650627613067627

In [ ]:
total=0
correct=0
for _,(input,label) in enumerate(test_loader):
  input=input.to(device)
  label=label.to(device)
  output=model(input)
  _,predicted=torch.max(output,1)
  correct+=(label==predicted).sum()
  total+=label.size(0)
print(correct/total*100)

tensor(7.3400, device='cuda:0')
